<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>


# Interacting with Tables in Storage

## Azure Actions Covered

* 

In this lecture, we're going to take a look at how to interact with tables in storage containers via the Python SDK.

To begin, we import our usual libraries as well as any useful environment variables (e.g. `AZURE_SUBSCRIPTION_ID`). 

In [1]:
# For generating storage account name
import random
from datetime import datetime

from azure.identity import AzureCliCredential, DefaultAzureCredential, InteractiveBrowserCredential
from azure.mgmt.resource import ResourceManagementClient
# New import
from azure.mgmt.storage import StorageManagementClient
# Table imports
from azure.data.tables import TableServiceClient, TableClient

from settings import AZURE_SUBSCRIPTION_ID

In [2]:
credential = AzureCliCredential()

In [3]:
table_service_client = TableServiceClient(
    endpoint='https://benbstorage1234.table.core.windows.net',
    credential=credential
)

In [5]:
table_client = table_service_client.create_table(table_name='bensTable')

In [6]:
table_client.url

'https://benbstorage1234.table.core.windows.net'

In [9]:
entity = table_client.create_entity(
    entity={
        'PartitionKey': 'Doe, John',
        'RowKey': '00001',
        'AdmissionDate': datetime(2023, 5, 10),
        'DischargeDate': datetime(2023, 5, 15),
        'Age': 78,
        'Diagnosis': 'Flu'
    }
)

In [10]:
for entity in table_client.list_entities():
    print(entity)

{'PartitionKey': 'Doe, John', 'RowKey': '00001', 'AdmissionDate': TablesEntityDatetime(2023, 5, 10, 6, 0, tzinfo=datetime.timezone.utc), 'DischargeDate': TablesEntityDatetime(2023, 5, 15, 6, 0, tzinfo=datetime.timezone.utc), 'Age': 78, 'Diagnosis': 'Flu'}


In [13]:
parameters = {
    'admission_date': datetime(2023, 5, 1)
}
query_filter = "AdmissionDate ge @admission_date"
result = table_client.query_entities(
    query_filter=query_filter,
    parameters=parameters
)
for item in result:
    print(item)

{'PartitionKey': 'Doe, John', 'RowKey': '00001', 'AdmissionDate': TablesEntityDatetime(2023, 5, 10, 6, 0, tzinfo=datetime.timezone.utc), 'DischargeDate': TablesEntityDatetime(2023, 5, 15, 6, 0, tzinfo=datetime.timezone.utc), 'Age': 78, 'Diagnosis': 'Flu'}


In [14]:
parameters = {
    'pk': 'Doe, John'
}
query_filter = "PartitionKey eq @pk"
result = table_client.query_entities(
    query_filter=query_filter,
    parameters=parameters
)
for item in result:
    print(item)

{'PartitionKey': 'Doe, John', 'RowKey': '00001', 'AdmissionDate': TablesEntityDatetime(2023, 5, 10, 6, 0, tzinfo=datetime.timezone.utc), 'DischargeDate': TablesEntityDatetime(2023, 5, 15, 6, 0, tzinfo=datetime.timezone.utc), 'Age': 78, 'Diagnosis': 'Flu'}


In [15]:
table_client.delete_entity(
    partition_key='Doe, John',
    row_key='00001'
)

In [16]:
for entity in table_client.list_entities():
    print(entity)

In [18]:
for table in table_service_client.list_tables():
    print(table.name)

bensTable


In [19]:
parameters = {
    'table_name': 'bensTable'
}
query_filter = 'TableName eq @table_name'
result = table_service_client.query_tables(
    query_filter=query_filter,
    parameters=parameters
)
for table in result:
    print(table.name)

bensTable


In [20]:
table_service_client.delete_table(table_name='bensTable')